In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()

In [7]:
# source: https://courses.geemap.org/geemap_intro/10_ts_inspector/ 
# (used to fix the errors in the next code cell, but later worked without this one)
if Map.user_roi is None:
    roi = ee.Geometry.Polygon(
        [
            [
                [-99.924088, 46.87005],
                [-99.924088, 47.092566],
                [-99.41391, 47.092566],
                [-99.41391, 46.87005],
                [-99.924088, 46.87005],
            ]
        ]
    )
else:
    roi = Map.user_roi

In [3]:
# Create annual composite of NAIP imagery
## select 4-band NAIP imagery
naip_ts = geemap.naip_timeseries(start_year=2009, end_year=2018)

## error1: "AttributeError: type object 'List' has no attribute 'sequence'"
## error2: "Exception: type object 'List' has no attribute 'sequence'"

## solution: add "roi" in the paranthesis
## solution2: removed roi and has no errors?

In [4]:
# create a list of layer names to be shown under the dropdown list
layer_names = ["NAIP " + str(year) for year in range(2009, 2019)]
print(layer_names)

['NAIP 2009', 'NAIP 2010', 'NAIP 2011', 'NAIP 2012', 'NAIP 2013', 'NAIP 2014', 'NAIP 2015', 'NAIP 2016', 'NAIP 2017', 'NAIP 2018']


In [5]:
# set visualization parameters
naip_vis = {"bands": ["N", "R", "G"]}

In [6]:
# Create a split-panel map
Map = geemap.Map()
Map.split_map()
Map.ts_inspector(
    left_ts=naip_ts,
    right_ts=naip_ts,
    left_names=layer_names,
    right_names=layer_names,
    left_vis=naip_vis,
    right_vis=naip_vis,
    # width = '110px'
)
Map
# Not every location has data for every year

## split panel map did not show up (with roi in the naip_timeseries cell)?
    ## solution: either add "Map.split_map()" in this code cell or "Map = geemap.Map()" at the very beginning (i.e. before specifying naip_ts=...)
## took very long to run
## data is huge, and it may take some time to load; need to zoom in to US CONUS in order for the NAIP layers to show up

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=Layo…

In [7]:
# Create annual composite of Landsat imagery
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
# draw an roi
region = Map.draw_last_feature
roi = region.geometry()

In [9]:
## can also define an roi
# if region is not None:
#     roi = region.geometry()
# else:
#     roi = ee.Geometry.Polygon(
#         [
#             [
#                 [-115.897448, 35.640766],
#                 [-115.897448, 36.603608],
#                 [-113.784915, 36.603608],
#                 [-113.784915, 35.640766],
#                 [-115.897448, 35.640766],
#             ]
#         ],
#         None,
#         False,
#     )

In [10]:
print(roi.getInfo())

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-78.971505, 35.963891], [-78.971505, 36.041937], [-78.844772, 36.041937], [-78.844772, 35.963891], [-78.971505, 35.963891]]]}


In [11]:
landsat_ts = geemap.landsat_timeseries(
    roi=roi, start_year=1984, end_year=2019, start_date="01-01", end_date="12-31")

In [12]:
layer_names = ["Landsat " + str(year) for year in range(1984, 2020)]
print(layer_names)

['Landsat 1984', 'Landsat 1985', 'Landsat 1986', 'Landsat 1987', 'Landsat 1988', 'Landsat 1989', 'Landsat 1990', 'Landsat 1991', 'Landsat 1992', 'Landsat 1993', 'Landsat 1994', 'Landsat 1995', 'Landsat 1996', 'Landsat 1997', 'Landsat 1998', 'Landsat 1999', 'Landsat 2000', 'Landsat 2001', 'Landsat 2002', 'Landsat 2003', 'Landsat 2004', 'Landsat 2005', 'Landsat 2006', 'Landsat 2007', 'Landsat 2008', 'Landsat 2009', 'Landsat 2010', 'Landsat 2011', 'Landsat 2012', 'Landsat 2013', 'Landsat 2014', 'Landsat 2015', 'Landsat 2016', 'Landsat 2017', 'Landsat 2018', 'Landsat 2019']


In [13]:
landsat_vis = {
    "min": 0,
    "max": 0.3,
    "gamma": [1, 1, 1],
    "bands": ["NIR", "Red", "Green"],
}

In [14]:
Map = geemap.Map()
Map.ts_inspector(
    left_ts=landsat_ts,
    right_ts=landsat_ts,
    left_names=layer_names,
    right_names=layer_names,
    left_vis=landsat_vis,
    right_vis=landsat_vis,
)
Map.centerObject(roi, zoom=8)
Map

# successfully ran without the split_map line

Map(center=[36.00291311978613, -78.90813850000106], controls=(WidgetControl(options=['position', 'transparent_…

In [31]:
# Compare Landsat imagery and NLCD
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [32]:
# draw an roi
region = Map.draw_last_feature
roi = region.geometry()

## error: NoneType is not callable
    ## solution: rerun the geemap.Map() line and redraw the polygon

In [33]:
print(roi.getInfo())

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-79.25113, 35.591212], [-79.25113, 36.198628], [-78.377558, 36.198628], [-78.377558, 35.591212], [-79.25113, 35.591212]]]}


In [17]:
# get the NLDC image collection
NLCD = ee.ImageCollection("USGS/NLCD")

In [18]:
# see how many layers are within this data collection
NLCD_layers = NLCD.aggregate_array("system:id").getInfo()
print(NLCD_layers)

['USGS/NLCD/NLCD1992', 'USGS/NLCD/NLCD2001', 'USGS/NLCD/NLCD2001_AK', 'USGS/NLCD/NLCD2001_HI', 'USGS/NLCD/NLCD2001_PR', 'USGS/NLCD/NLCD2004', 'USGS/NLCD/NLCD2006', 'USGS/NLCD/NLCD2008', 'USGS/NLCD/NLCD2011', 'USGS/NLCD/NLCD2011_AK', 'USGS/NLCD/NLCD2011_HI', 'USGS/NLCD/NLCD2011_PR', 'USGS/NLCD/NLCD2013', 'USGS/NLCD/NLCD2016']


In [20]:
# select specific layers
NLCD2001 = ee.Image("USGS/NLCD/NLCD2001").select("landcover")
NLCD2006 = ee.Image("USGS/NLCD/NLCD2006").select("landcover")
NLCD2011 = ee.Image("USGS/NLCD/NLCD2011").select("landcover")
NLCD2016 = ee.Image("USGS/NLCD/NLCD2016").select("landcover")

In [21]:
NLCD = ee.ImageCollection([NLCD2001, NLCD2006, NLCD2011, NLCD2016])

In [22]:
NLCD_layer_names = ["NLCD " + str(year) for year in range(2001, 2017, 5)]
print(NLCD_layer_names)

['NLCD 2001', 'NLCD 2006', 'NLCD 2011', 'NLCD 2016']


In [23]:
# roi = ee.Geometry.Polygon(
#     [
#         [
#             [-115.897448, 35.640766],
#             [-115.897448, 36.603608],
#             [-113.784915, 36.603608],
#             [-113.784915, 35.640766],
#             [-115.897448, 35.640766],
#         ]
#     ],
#     None,
#     False,
# )

In [34]:
landsat_ts = geemap.landsat_timeseries(
    roi=roi, start_year=2001, end_year=2016, start_date="01-01", end_date="12-31"
)

In [35]:
landsat_layer_names = ["Landsat " + str(year) for year in range(2001, 2017)]

In [36]:
landsat_vis = {
    "min": 0,
    "max": 0.3,
    "gamma": [1, 1, 1],
    "bands": ["NIR", "Red", "Green"],
}

In [37]:
nlcd_vis = {"bands": ["landcover"]}

In [38]:
Map = geemap.Map()
Map.ts_inspector(
    left_ts=landsat_ts,
    right_ts=NLCD,
    left_names=landsat_layer_names,
    right_names=NLCD_layer_names,
    left_vis=landsat_vis,
    right_vis=nlcd_vis,
)
Map.centerObject(roi, zoom=8)
Map

## the NLCD layers in the final split map still cover the entire US even though roi has been specified ahead of time
    ## in this case, what is the purpose of specifying roi when setting up NLCD time series?

Map(center=[35.89479520715624, -78.81434399999993], controls=(WidgetControl(options=['position', 'transparent_…